In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

tfd = tfp.distributions
tfpl = tfp.layers

# Simulated time series data
def generate_data(n_samples=500, time_steps=10):
    X = np.linspace(0, 100, n_samples)
    y = np.sin(X / 10) + np.random.normal(0, 0.1, n_samples)
    X_seq, y_seq = [], []
    for i in range(len(X) - time_steps):
        X_seq.append(y[i:i + time_steps])
        y_seq.append(y[i + time_steps])
    return np.array(X_seq)[..., np.newaxis], np.array(y_seq)

# Prepare data
time_steps = 20
X, y = generate_data(n_samples=1000, time_steps=time_steps)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define Bayesian LSTM model
def build_bayesian_lstm(input_shape):
    model = tf.keras.Sequential([
        tfpl.LSTMFlipout(32, activation='tanh', input_shape=input_shape),
        tfpl.DenseFlipout(1)
    ])
    model.compile(
        loss='mean_squared_error',
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.01)
    )
    return model

# Train model
model = build_bayesian_lstm((time_steps, 1))
model.summary()
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.1)

# Predict with uncertainty
def predict_with_uncertainty(f_model, X, n_iter=100):
    predictions = np.array([f_model(X, training=True).numpy().squeeze() for _ in range(n_iter)])
    pred_mean = predictions.mean(axis=0)
    pred_std = predictions.std(axis=0)
    return pred_mean, pred_std

mean_preds, std_preds = predict_with_uncertainty(model, X_test)

# Plot results
plt.figure(figsize=(12, 5))
plt.plot(y_test, label='True')
plt.plot(mean_preds, label='Predicted Mean')
plt.fill_between(np.arange(len(y_test)), mean_preds - 2*std_preds, mean_preds + 2*std_preds,
                 color='gray', alpha=0.3, label='Uncertainty (±2σ)')
plt.legend()
plt.title('Bayesian LSTM Prediction with Uncertainty')
plt.xlabel('Sample')
plt.ylabel('Value')
plt.show()


ModuleNotFoundError: No module named 'tensorflow_probability'

In [3]:
pip install tensorflow tensorflow-probability

  Using cached typing_extensions-4.5.0-py3-none-any.whl.metadata (8.5 kB)
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   --------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 8.30.0 requires typing-extensions>=4.6; python_version < "3.12", but you have typing-extensions 4.5.0 which is incompatible.
pydantic 2.10.3 requires typing-extensions>=4.12.2, but you have typing-extensions 4.5.0 which is incompatible.
pydantic-core 2.27.1 requires typing-extensions!=4.7.0,>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
sqlalchemy 2.0.37 requires typing-extensions>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
torch 2.6.0 requires typing-extensions>=4.10.0, but you have typing-extensions 4.5.0 which is incompatible.
